## Feature extraction and listing image paths

In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.feature import hog
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from lesson_functions import *
from IPython.display import HTML
%matplotlib inline

cars = glob.glob('/data/vehicles/*/*.png')
notcars = glob.glob('/data/non-vehicles/*/*.png')

# Reduce the sample size because
# The quiz evaluator times out after 13s of CPU time
sample_size = 500
cars = cars[0:sample_size]
notcars = notcars[0:sample_size]

Here is an example using HOG parameters of orientations=8, pixels_per_cell=(8, 8) and cells_per_block=(2,2)